In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import torch
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import skew, spearmanr
import shap
from sklearn.feature_selection import RFE
from collections import Counter
from scipy.spatial.distance import euclidean
from sklearn.metrics import silhouette_score

from sklearn.model_selection import train_test_split, KFold, LeavePGroupsOut, LeaveOneGroupOut, LeaveOneOut
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from scipy.spatial.distance import cdist


from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from tabpfn import TabPFNRegressor
from lightgbm import LGBMRegressor
import lightgbm
from sklearn_extra.cluster import KMedoids 


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import random
seed = random.randint(0, 100000)
np.random.seed(seed)
torch.manual_seed(seed)


In [2]:
data = pd.read_csv('all-features-imputed-v2.csv')

#display(data)
#display(data.columns.to_list())


acceleration    = data.filter(like='acceleration').columns.tolist()
heartrate       = data.filter(like='heartrate').filter(regex='^(?!.*sleep)').columns.tolist()
motion          = data.filter(like='motion').columns.tolist()
position        = data.filter(like='position').columns.tolist()
sleep           = data.filter(like='sleep').columns.tolist()
step            = data.filter(like='step').columns.tolist()
demographics    = ['age','sex']







In [3]:
modalities = acceleration + heartrate + motion + position + sleep + step + demographics

sensor = data[modalities]

ohs = data['ohs']
sis = data['sis']

participant = data['participant']

x = np.array(sensor)
#Set target variable
y = np.array(ohs)
p = np.array(participant)



In [ ]:
Y_TRUES = np.empty([0])
Y_PREDS = np.empty([0])
SHAP = []
X_TEST = []

cv = KFold(n_splits=5, shuffle=True, random_state=seed)
for fold, (train_idx, test_idx) in enumerate(cv.split(x), start=1):
    
    print(f"Fold {fold}: train={len(train_idx)} test={len(test_idx)}")

    x_train, x_test = x[train_idx], x[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    normalizer = MinMaxScaler()
    x_train = normalizer.fit_transform(x_train)
    x_test = normalizer.transform(x_test)

    model = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.1,
        depth=3,
        loss_function='RMSE',
        verbose=False
    )

    model.fit(x_train, y_train, eval_set=(x_train, y_train), use_best_model=True, early_stopping_rounds=100)
    y_preds = model.predict(x_test)

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(x_test)

    SHAP.append(shap_values)
    X_TEST.append(x_test)

    Y_TRUES = np.append(Y_TRUES, y_test)
    Y_PREDS = np.append(Y_PREDS, y_preds)


shap_df = pd.DataFrame({
    'feature': modalities,
    'mean_abs_shap': np.abs(np.vstack(SHAP)).mean(axis=0).round(4)
}).sort_values(by='mean_abs_shap', ascending=False)
# display(shap_df)
#shap.summary_plot(np.vstack(SHAP), pd.DataFrame(np.vstack(X_TEST), columns=modalities), max_display=24)



# Select the first num_features features based on SHAP importance + demographics
num_features = 16
x = np.array(data[
    shap_df['feature'].iloc[:num_features].to_list()
     + demographics
    ])




In [5]:

biweekly_scores = y.reshape(-1, 7)[:, 0]  # Shape: (80 weeks, 7 day)
weekly_bags = x.reshape((80, 7, -1))







In [6]:

def compute_connectivity(distance_matrix, cluster_labels, k_neighbors=5):
    """
    Compute connectivity score for clustering results.
    Lower values indicate better connectivity (0 is perfect).
    """
    n_samples = distance_matrix.shape[0]
    
    # Get k nearest neighbors for each point (excluding self)
    neighbor_indices = np.argsort(distance_matrix, axis=1)[:, 1:k_neighbors+1]  # Exclude self
    
    connectivity = 0
    for i in range(n_samples):
        for neighbor in neighbor_indices[i]:
            if cluster_labels[i] != cluster_labels[neighbor]:
                connectivity += 1
                
    # Normalize to [0,1] range
    max_possible = n_samples * k_neighbors
    return connectivity / max_possible if max_possible > 0 else 0



def compute_distance_matrix(bags): #OBS CHECK VALIDITY
    n= len(bags)
    #print("BAGS:",bags)
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1,n):
            dist = average_hausdorff_distance(bags[i], bags[j])
            matrix[i, j] = matrix[j, i] = dist

    return matrix

def clustering(bags,matrix, n_clusters=10):#OBS MAYBE KMEANS???
    """Perform Clustering using K-medoids"""
    kmedoids=KMedoids(n_clusters=n_clusters, metric='precomputed', method='pam', init="build")
    #print(kmedoids)
                
    clusters = kmedoids.fit_predict(matrix)
    silhouette_avg = silhouette_score(matrix, clusters, metric="precomputed")
    connectivity = compute_connectivity(matrix, clusters)

    #print(f"Silhouette Score: {silhouette_avg:.3f}")
    valid_medoids = [idx for idx in kmedoids.medoid_indices_ if idx < len(bags)]
    medoids = [bags[idx] for idx in valid_medoids]

    return medoids, silhouette_avg , connectivity



def BARTMIP_transform(bags, medoids):#OBS CHECK VALIDITY
    """Transform bags using BARTMIP"""
    transformed = []
    for bag in bags:
        dist_features = []
        for medoid in medoids:
            # Multiple distance measures
            min_dist = np.min(cdist(bag, medoid, 'euclidean'))
            avg_dist = np.mean(np.min(cdist(bag, medoid, 'euclidean'), axis=1))
            hausdorff = average_hausdorff_distance(bag, medoid)
            dist_features.extend([min_dist, avg_dist, hausdorff])
        transformed.append(dist_features)


    return np.array(transformed)




def average_hausdorff_distance(bag1, bag2):
    """Calculate average Hausdorff distance between two 2D arrays"""
    bag1 = bag1[~np.isnan(bag1).any(axis=1)]
    bag2 = bag2[~np.isnan(bag2).any(axis=1)]
    bag1 = bag1[np.isfinite(bag1).all(axis=1)]
    bag2 = bag2[np.isfinite(bag2).all(axis=1)]
    
    
    
    #dist_matrix = cdist(bag1, bag2, metric='euclidean')
    min_dists_a = [min(euclidean(a, b) for b in bag2) for a in bag1]
    min_dists_b = [min(euclidean(b, a) for a in bag1) for b in bag2]
    #if np.isnan((avg_dist1 + avg_dist2) / 2):
        #print(avg_dist1,avg_dist2)
    return (np.mean(min_dists_a) + np.mean(min_dists_b)) / 2



In [7]:
def get_scaling_param(x_train):
    """Set Scaling paramers """
 
        
    mean = np.mean(x_train, axis=0)
    std = np.std(x_train, axis=0)
    std[std == 0] = 1 
    
    x_standardized = (x_train - mean) / std
    
    min_val = np.min(x_standardized, axis=0)
    max_val = np.max(x_standardized, axis=0)
    range_val = max_val - min_val
    range_val[range_val == 0] = 1  
    
    return {
        'mean': mean,
        'std': std,
        'min': min_val,
        'range': range_val
    }


def apply_scaling(x, scaling_param):
    """
    Flatten and scales data"""
    original_shape = x.shape
    if len(x.shape) > 2:
        x_flat = x.reshape(-1, x.shape[-1])
    else:
        x_flat = x
    
    x_scaled = (x_flat - scaling_param['mean']) / scaling_param['std']
    
    x_scaled = (x_scaled - scaling_param['min']) / scaling_param['range']
    
    # Reshape back to original shape if needed
    if len(original_shape) > 2:
        x_scaled = x_scaled.reshape(original_shape)
    
    return x_scaled


def cross(x, y, n_clusters=10):
    mae_scores = []
    r2_scores = []
    silhouette_scores = []
    connectivity_scores = []

    cv = KFold(n_splits=5, shuffle=True, random_state=seed)
    counter=0
    for train_idx, test_idx in cv.split(x):
        counter += 1
        print(f"Processing fold {counter}/5")
        x_train, x_test = x[train_idx], x[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]


        # Flatten the data for scaling
        x_train_flat = x_train.reshape(-1, x_train.shape[-1])
        
        # Get scaling values from training data
        scaling_values = get_scaling_param(x_train_flat)
        
        # Apply scaling to both training and test data
        x_train_scaled = apply_scaling(x_train, scaling_values)
        x_test_scaled = apply_scaling(x_test, scaling_values)

        # Compute distance matrix
        distance_matrix = compute_distance_matrix(x_train_scaled)
        
        # Perform clustering
        medoids, sil_score, connectivity = clustering(x_train_scaled, distance_matrix, n_clusters)
        silhouette_scores.append(sil_score)
        connectivity_scores.append(connectivity)
        
        # Transform data using medoids
        x_train_transformed = BARTMIP_transform(x_train_scaled, medoids)
        x_test_transformed = BARTMIP_transform(x_test_scaled, medoids)
        
        # Train and evaluate model
        model = CatBoostRegressor(
            iterations=1000,
            learning_rate=0.1,
            depth=3,
            loss_function='RMSE',
            verbose=False
        )
        model.fit(
            x_train_transformed, y_train,
            eval_set=(x_test_transformed, y_test),
            use_best_model=True,
            early_stopping_rounds=100
        )
        y_pred = model.predict(x_test_transformed)
        
        # Calculate metrics
        mae_scores.append(mean_absolute_error(y_test, y_pred))
        r2_scores.append(r2_score(y_test, y_pred))
        print(r2_score(y_test, y_pred))
    
    # Return mean metrics with cluster info
    return {
        'n_clusters': n_clusters,
        'mean_mae': np.mean(mae_scores),
        'mean_r2': np.mean(r2_scores),
        'mean_silhouette': np.mean(silhouette_scores),
        'mean_connectivity': np.mean(connectivity_scores)
    }

In [8]:








def loso(x, y, n_clusters=10):
    mae_scores = []
    silhouette_scores = []
    all_y_true = []
    all_y_pred = []
    connectivity_scores = []
    
    cv = LeaveOneOut()
    counter= 0
    for train_idx, test_idx in cv.split(x):
        counter += 1
        print(f"Processing fold {counter}/80")
        x_train, x_test = x[train_idx], x[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        

        
        # Get scaling values from training data
        scaling_values = get_scaling_param(x_train.reshape(-1, x_train.shape[-1]))
        
        # Apply scaling to both training and test data
        x_train = apply_scaling(x_train, scaling_values)
        x_test = apply_scaling(x_test, scaling_values)
        
        # Compute distance matrix 
        distance_matrix = compute_distance_matrix(x_train)
        
        # Perform clustering
        medoids, sil_score,connectivity = clustering(x_train, distance_matrix, n_clusters)
        silhouette_scores.append(sil_score)
        connectivity_scores.append(connectivity)
        
        # Transform data using medoids
        x_train_transformed = BARTMIP_transform(x_train, medoids)
        x_test_transformed = BARTMIP_transform(x_test, medoids)
        
        # Train and evaluate model
        model = CatBoostRegressor(
            iterations=1000,
            learning_rate=0.1,
            depth=3,
            loss_function='RMSE',
            verbose=False
        )
        model.fit(
            x_train_transformed, y_train,
            eval_set=(x_test_transformed, y_test),
            use_best_model=True,
            early_stopping_rounds=100
        )
        y_pred = model.predict(x_test_transformed)
        
        # Collect all predictions and true values
        all_y_true.append(y_test[0])
        all_y_pred.append(y_pred[0])
        mae_scores.append(mean_absolute_error(y_test, y_pred))
    
    # Calculate global metrics
    global_r2 = r2_score(all_y_true, all_y_pred)
    
    return {
        'n_clusters': n_clusters,
        'mean_mae': np.mean(mae_scores),
        'mean_r2': global_r2,
        'mean_silhouette': np.mean(silhouette_scores),
        'mean_connectivity': np.mean(connectivity_scores)
    }


In [ ]:


results = []


results.append(loso(weekly_bags, biweekly_scores, n_clusters=13))

sorted_results = sorted(results, key=lambda x: x['mean_mae'])

# Print formatted results
print("Rank | Clusters | Mean MAE | Mean R² | Mean Silhouette")
print("-------------------------------------------------------")
for idx, result in enumerate(sorted_results):
    print(f"{idx+1:4} | {result['n_clusters']:8} | {result['mean_mae']:.4f} | {result['mean_r2']:.4f} | {result['mean_silhouette']:.4f} | {result['mean_connectivity']:.4f}")